In [ ]:
# load data from mongodb regarding
# commit message, fileactions, lines changed, change type (added, deleted, modified)
import pandas as pd
from bson.objectid import ObjectId
from mongoengine import connect
from pycoshark.mongomodels import Project, VCSSystem, Commit, Tag, File, CodeEntityState, FileAction, People, IssueSystem, Issue, Message, MailingList, Event, MynbouData, Identity, Hunk, Branch
loc = {'host': '127.0.0.1',
       'port': 27017,
       'db': 'smartshark',
       'username': 'root',
       'password': '',
       'authentication_source': 'smartshark',
       'connect': False}
connect(**loc)

In [ ]:
dfall = pd.read_pickle('../data/aggregated_full.pickle')
len(dfall)

In [ ]:
dfall['commit_msg'] = ''
dfall['files_added'] = 0
dfall['files_deleted'] = 0
dfall['files_modified'] = 0

for project_name in dfall['project'].unique():
    pr = Project.objects.get(name=project_name)
    vcs = VCSSystem.objects.get(project_id=pr.id)
    
    for revision_hash in dfall[dfall['project'] == project_name]['revision'].unique():
        # fetch commit message
        files_added = 0
        files_deleted = 0
        files_modified = 0
        c = Commit.objects.get(vcs_system_id=vcs.id, revision_hash=revision_hash)
        
        for fa in FileAction.objects.filter(commit_id=c.id):
            if fa.mode == 'M':
                files_modified += 1
            elif fa.mode in ['A', 'C']:
                files_added += 1
            elif fa.mode == 'D':
                files_deleted += 1
        
        f = dfall[(dfall['project'] == project_name) & (dfall['revision'] == revision_hash)]
        dfall.loc[f.index, 'commit_msg'] = c.message
        dfall.loc[f.index, 'files_added'] = files_added
        dfall.loc[f.index, 'files_modified'] = files_modified
        dfall.loc[f.index, 'files_deleted'] = files_deleted
        

In [ ]:
dfall.to_pickle('../data/aggregated_full_enriched.pickle')

In [ ]:
dfall.head()

In [ ]:
# count complete commits our study subjects have
commits = 0
for project_name in dfall['project'].unique():
    p = Project.objects.get(name=project_name)
    vcs = VCSSystem.objects.get(project_id=p.id)
    
    commits += Commit.objects.filter(vcs_system_id=vcs.id).count()

print('study subjects have', commits, 'number of commits')